In [1]:
%pylab inline
%matplotlib inline

import os
import time
import collections
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("/Users/Bya/git/predictEPL/utils/")
sys.path.append("/Users/Bya/git/predictEPL/config/")
sys.path.append("/Users/Bya/git/predictEPL/WebScrapping/")

import paths
import espn_urls
import my_plot
import scrap_espn_gamecast
import emolex
import useful_methods

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Game Infos
os.chdir(paths.READ_PATH_GAME_INFO)
dfGameInfos = useful_methods.csv_dic_df('game_infos.csv')

# Convert number strings to integers
dfGameInfos['GW'] = [int(GW) for GW in dfGameInfos['GW']]
dfGameInfos['score_ht_home'] = [int(number) for number in dfGameInfos['score_ht_home']]
dfGameInfos['score_ht_away'] = [int(number) for number in dfGameInfos['score_ht_away']]
dfGameInfos['score_ft_home'] = [int(number) for number in dfGameInfos['score_ft_home']]
dfGameInfos['score_ft_away'] = [int(number) for number in dfGameInfos['score_ft_away']]

dfGameInfos.head()

,GW,away_team,date,home_team,score_ft_away,score_ft_home,score_ht_away,score_ht_home,time
0,1,Tottenham,Sat. 8 Aug.,United,0,1,0,1,11:45
1,1,Watford,Sat. 8 Aug.,Everton,2,2,1,0,14:00
2,1,Sunderland,Sat. 8 Aug.,Leicester,2,4,0,3,14:00
3,1,Crystal,Sat. 8 Aug.,Norwich,3,1,1,0,14:00
4,1,Villa,Sat. 8 Aug.,Bournemouth,1,0,0,0,14:00


In [32]:
# HT: Equal games
# dfGameInfosEqual = dfGameInfos[(dfGameInfos['score_ht_home'] == dfGameInfos['score_ht_away']) &
#                                (dfGameInfos['GW'] >= 4)].copy()

weeks = list(dfGameInfos['GW'])
home_teams = list(dfGameInfos['home_team'])
away_teams = list(dfGameInfos['away_team'])
score_ht_homes = list(dfGameInfos['score_ht_home'])
score_ht_aways = list(dfGameInfos['score_ht_away'])
score_ft_homes = list(dfGameInfos['score_ft_home'])
score_ft_aways = list(dfGameInfos['score_ft_away'])

In [4]:
# Read Emotion-Lexicon-Soccer as Dictionary
dic_emolex_soccer, dic_emolex_stemmed_soccer = emolex.EmolexSoccerDic()

All Words: 14163


In [5]:
def EmolexPN(df, start=1, end=63):
    dfEmolex = df.copy()
    
    # set interval
    ith_minute_start = start
    ith_minute_end = end

    # convert to integer
    dfEmolex['ith_minute'] = [int(number) for number in dfEmolex['ith_minute']]
    dfEmolex['positive'] = [int(number) for number in dfEmolex['positive']]
    dfEmolex['negative'] = [int(number) for number in dfEmolex['negative']]

    # Minute Interval's DF
    dfEmolex = dfEmolex[
        (dfEmolex['ith_minute'] >= ith_minute_start) &
        (dfEmolex['ith_minute'] <= ith_minute_end)
    ].copy()

    # Counting pos, neg
    positive = sum(list(dfEmolex['positive']))
    negative = sum(list(dfEmolex['negative']))
    
    # Return Positve Percentage
    return (positive / (positive + negative))

In [33]:
res_ht_rt = []

start_time = time.time()

for index in range(len(weeks)):
    try:
        # Read Single Game as DF
        filename = home_teams[index] + '_vs_' + away_teams[index] + '.csv'
        os.chdir(paths.READ_PATH_EXTRACTED_CSV + 'GW' + str(weeks[index]) + '/SingleGames/')
        df = useful_methods.csv_dic_df(filename)

        # Filter DF: remove tweets of stream, bots ...
        dfFilter = useful_methods.FilterDF(df)

        # Count Emolex cateries on Tweets by minutes
        dfFilterEmolexSoccerHome = emolex.CreateEmolexDF(dfFilter[dfFilter['side'] == 'home'], dic_emolex_soccer, dic_emolex_stemmed_soccer)
        dfFilterEmolexSoccerAway = emolex.CreateEmolexDF(dfFilter[dfFilter['side'] == 'away'], dic_emolex_soccer, dic_emolex_stemmed_soccer)

        # Calculate PN %
        emolex_pn_home = EmolexPN(dfFilterEmolexSoccerHome)
        emolex_pn_away = EmolexPN(dfFilterEmolexSoccerAway)

        # Append Resulst
        res_ht_rt.append([index, emolex_pn_home, emolex_pn_away])
    except:
        continue

print("[Done]: %.2f" % (time.time() - start_time))

[Done]: 864.43


/Users/Bya/.virtualenvs/py3/lib/python3.4/site-packages/pandas/core/ops.py:680: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


In [34]:
ok = 0
ng = 0
eq = 0

for res in res_ht_rt:
    index, pn_home, pn_away = res
    
    if (pn_home > pn_away) and (score_ft_homes[index] > score_ft_aways[index]):
        ok += 1
        print("\n------------------------------")
        print("[GAME]: GW%s %s vs %s" %(weeks[index], home_teams[index], away_teams[index]))
        print("[SCORE]: HT(%s-%s) => FT(%s-%s)" % (score_ht_homes[index], score_ht_aways[index],
                                          score_ft_homes[index], score_ft_aways[index]))
        print("\t[Emolex PN Score]: %s vs %s" % (pn_home, pn_away))

    elif (pn_home < pn_away) and (score_ft_homes[index] < score_ft_aways[index]):
        ok += 1
        print("\n------------------------------")
        print("[GAME]: GW%s %s vs %s" %(weeks[index], home_teams[index], away_teams[index]))
        print("[SCORE]: HT(%s-%s) => FT(%s-%s)" % (score_ht_homes[index], score_ht_aways[index],
                                          score_ft_homes[index], score_ft_aways[index]))
        print("[Emolex PN Score]: %s vs %s" % (pn_home, pn_away))
    
    elif score_ft_homes[index] == score_ft_aways[index]:
        eq += 1
    else:
        ng += 1

print("\n\n\nOK: %s, NG: %s, EQ: %s" % (ok, ng, eq))


------------------------------
[GAME]: GW4 Newcastle vs Arsenal
[SCORE]: HT(0-0) => FT(0-1)
[Emolex PN Score]: 0.364154723378 vs 0.470665499124

------------------------------
[GAME]: GW4 Stoke vs WestBromwich
[SCORE]: HT(0-1) => FT(0-1)
[Emolex PN Score]: 0.134433962264 vs 0.509881422925

------------------------------
[GAME]: GW4 Liverpool vs WestHam
[SCORE]: HT(0-2) => FT(0-3)
[Emolex PN Score]: 0.545269394557 vs 0.816371681416

------------------------------
[GAME]: GW4 Chelsea vs Crystal
[SCORE]: HT(0-0) => FT(1-2)
[Emolex PN Score]: 0.582383419689 vs 0.837067209776

------------------------------
[GAME]: GW4 City vs Watford
[SCORE]: HT(0-0) => FT(2-0)
	[Emolex PN Score]: 0.751351351351 vs 0.702702702703

------------------------------
[GAME]: GW4 Southampton vs Norwich
[SCORE]: HT(1-0) => FT(3-0)
	[Emolex PN Score]: 0.619572708477 vs 0.485772357724

------------------------------
[GAME]: GW4 Swansea vs United
[SCORE]: HT(0-0) => FT(2-1)
	[Emolex PN Score]: 0.801411290323 vs 0.61

In [110]:
# Save Resulst as CSV
dfResult = dfGameInfos.copy()

def PnScore(index, side):
    for res in res_ht_rt:
        if index == res[0]:
            return res[side]
    return -1

dfResult['pn_away'] = [PnScore(index, 2) for index in range(len(weeks))]
dfResult['pn_home'] = [PnScore(index, 1) for index in range(len(weeks))]


# Save as CSV
useful_methods.DFtoCSV(dfResult, paths.READ_PATH_GAME_INFO, 'Predict-Emolex-63min', index=False)
print("[Saved in]: %s" % (paths.READ_PATH_GAME_INFO + 'Predict-Emolex-63min.csv'))

[Saved in]: /Users/Bya/Dropbox/Research/datas/EPL/Predict-Emolex-63min.csv


In [105]:
def PnAnalysis(dfResult):
    # Predicted: OK
    predict_good = len(dfResult[
        (dfResult['pn_home'] != -1) &

        # Predict: GOOD
        (
            # PN: Home > Away & FT: Home > Away
            ((dfResult['pn_home'] > dfResult['pn_away']) & (dfResult['score_ft_home'] > dfResult['score_ft_away'])) |
            # PN: Home < Away & FT: Home < Away
            ((dfResult['pn_home'] < dfResult['pn_away']) & (dfResult['score_ft_home'] < dfResult['score_ft_away']))
        )
    ])
    print("[PREDICTED GOOD]: %s" %(predict_good))

    # Predicted: NG
    predict_ng = len(dfResult[
        (dfResult['pn_home'] != -1) &

        # Predict:  NOT GOOD
        (
            # PN: Home > Away & FT: Home < Away
            ((dfResult['pn_home'] > dfResult['pn_away']) & (dfResult['score_ft_home'] < dfResult['score_ft_away'])) |
            # PN: Home < Away & FT: Home > Away
            ((dfResult['pn_home'] < dfResult['pn_away']) & (dfResult['score_ft_home'] > dfResult['score_ft_away']))
        )
    ])
    print("[PREDICTED NG]: %s" %(predict_ng))

    # FT : Equal
    ft_equal = len(dfResult[
        (dfResult['pn_home'] != -1) &

        # FT: Home == Away
        (dfResult['score_ft_home'] == dfResult['score_ft_away'])
    ])
    print("[Full-Time Equal]: %s" %(ft_equal))

    return predict_good, predict_ng, ft_equal


PnAnalysis(dfResult)
print("\n")

PnAnalysis(dfResult[(dfResult['score_ht_home'] == dfResult['score_ht_away'])])
print("\n")

PnAnalysis(dfResult[(dfResult['score_ht_home'] > dfResult['score_ht_away'])])
print("\n")

PnAnalysis(dfResult[(dfResult['score_ht_home'] < dfResult['score_ht_away'])])
print("\n")

[PREDICTED GOOD]: 59
[PREDICTED NG]: 26
[Full-Time Equal]: 31


[PREDICTED GOOD]: 19
[PREDICTED NG]: 15
[Full-Time Equal]: 20


[PREDICTED GOOD]: 25
[PREDICTED NG]: 8
[Full-Time Equal]: 8


[PREDICTED GOOD]: 15
[PREDICTED NG]: 3
[Full-Time Equal]: 3




In [113]:
dfResult[
        (dfResult['pn_home'] != -1) &
        
        # HT: Home == Away
        (dfResult['score_ht_home'] > dfResult['score_ht_away']) &
        
        # FT: Home == Away
        (dfResult['pn_home'] < dfResult['pn_away'])
    ]

,GW,away_team,date,home_team,score_ft_away,score_ft_home,score_ht_away,score_ht_home,time,pn_away,pn_home
34,4,Sunderland,Sat. 29 Aug.,Villa,2,2,1,2,14:00,1.000000,0.492228
35,4,Leicester,Sat. 29 Aug.,Bournemouth,1,1,0,1,14:00,0.628975,0.397810
78,8,Tottenham,Sun. 4 Oct.,Swansea,2,2,1,2,15:00,0.618241,0.465340
79,8,United,Sun. 4 Oct.,Arsenal,0,3,0,3,15:00,0.702616,0.604129
96,10,Newcastle,Sun. 25 Oct.,Sunderland,0,3,0,1,12:00,0.481673,0.386325
103,11,WestHam,Sat. 31 Oct.,Watford,0,2,0,1,15:00,0.678571,0.587912
143,15,Norwich,Sat. 5 Dec.,Watford,0,2,0,1,15:00,0.508124,0.439614
153,16,Swansea,Sat. 12 Dec.,City,1,2,0,1,15:00,0.812298,0.692842


In [117]:
dfResult[
        (dfResult['pn_home'] != -1) &
        
        # HT: Home == Away
        (dfResult['score_ht_home'] > dfResult['score_ht_away']) &
        
        # FT: Home == Away
        (dfResult['score_ft_home'] <= dfResult['score_ft_away']) &
    
        # PN: Home < Away
        (dfResult['pn_home'] < dfResult['pn_away'])
    ]

,GW,away_team,date,home_team,score_ft_away,score_ft_home,score_ht_away,score_ht_home,time,pn_away,pn_home
34,4,Sunderland,Sat. 29 Aug.,Villa,2,2,1,2,14:00,1.000000,0.492228
35,4,Leicester,Sat. 29 Aug.,Bournemouth,1,1,0,1,14:00,0.628975,0.397810
78,8,Tottenham,Sun. 4 Oct.,Swansea,2,2,1,2,15:00,0.618241,0.465340


In [118]:
dfResult[
        (dfResult['pn_home'] != -1) &
        
        # HT: Home == Away
        (dfResult['score_ht_home'] < dfResult['score_ht_away']) &
        
        # FT: Home == Away
        (dfResult['score_ft_home'] >= dfResult['score_ft_away']) &
    
        # PN: Home < Away
        (dfResult['pn_home'] > dfResult['pn_away'])
    ]

,GW,away_team,date,home_team,score_ft_away,score_ft_home,score_ht_away,score_ht_home,time,pn_away,pn_home
131,14,Everton,Sat. 28 Nov.,Bournemouth,3,3,2,0,15:00,0.669826,0.693498


In [84]:
dfResult[
    (dfResult['pn_home'] != -1) &
    
    # Reversed WIN
    (
        # HT: Home > Away & FT: Home < Away 
        ((dfResult['score_ht_home'] > dfResult['score_ht_away']) & (dfResult['score_ft_home'] < dfResult['score_ft_away'])) |
        
        # HT: Home < Away & FT: Home > Away 
        ((dfResult['score_ht_home'] < dfResult['score_ht_away']) & (dfResult['score_ft_home'] > dfResult['score_ft_away']))
    )        
]

,GW,away_team,date,home_team,score_ft_away,score_ft_home,score_ht_away,score_ht_home,time,pn_away,pn_home
48,5,Villa,Sun. 13 Sep.,Leicester,2,3,1,0,15:00,0.762478,0.636175
69,7,Everton,Mon. 28 Sep.,WestBromwich,3,2,0,1,19:00,0.487965,0.634315
102,11,Leicester,Sat. 31 Oct.,WestBromwich,3,2,0,1,15:00,0.600939,0.891566
158,16,Newcastle,Sun. 13 Dec.,Tottenham,2,1,0,1,16:00,0.587031,0.776910


In [85]:
dfResult[
    (dfResult['pn_home'] != -1) &
    
    # Deserved WIN
    (
        # HT: Home > Away & FT: Home > Away 
        ((dfResult['score_ht_home'] > dfResult['score_ht_away']) & (dfResult['score_ft_home'] > dfResult['score_ft_away'])) |
        
        # HT: Home < Away & FT: Home < Away 
        ((dfResult['score_ht_home'] < dfResult['score_ht_away']) & (dfResult['score_ft_home'] < dfResult['score_ft_away']))
    )        
]

,GW,away_team,date,home_team,score_ft_away,score_ft_home,score_ht_away,score_ht_home,time,pn_away,pn_home
31,4,WestBromwich,Sat. 29 Aug.,Stoke,1,0,1,0,14:00,0.509881,0.134434
32,4,WestHam,Sat. 29 Aug.,Liverpool,3,0,2,0,14:00,0.816372,0.545269
38,4,Norwich,Sun. 30 Aug.,Southampton,0,3,0,1,12:30,0.485772,0.619573
40,5,Chelsea,Sat. 12 Sep.,Everton,1,3,1,2,11:45,0.482983,0.597388
41,5,Bournemouth,Sat. 12 Sep.,Norwich,1,3,0,1,14:00,0.722488,0.767094
44,5,Stoke,Sat. 12 Sep.,Arsenal,0,2,0,1,14:00,0.695122,0.742400
49,5,Newcastle,Mon. 14 Sep.,WestHam,0,2,0,1,19:00,0.546692,0.732213
52,6,Sunderland,Sat. 19 Sep.,Bournemouth,0,2,0,2,14:00,0.497671,0.849576
53,6,WestBromwich,Sat. 19 Sep.,Villa,1,0,1,0,14:00,0.646934,0.519370
54,6,Watford,Sat. 19 Sep.,Newcastle,2,1,2,0,14:00,0.787342,0.504107
